In [ ]:
"""
Prepare Paolo et al. 2023 basal melt to use in summer paper

"""

In [14]:
import xarray as xr
import numpy as np

In [8]:
outputpath_raw = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/'
home_path = '/bettik/burgardc/'

In [9]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

In [4]:
inputpath_Paolo_processed = '/bettik/millanr/DATA_SERVER/ANTARCTICA/OCEANICE/DATASET_FINAL/'
integrated_Paolo = pd.read_csv(inputpath_Paolo_processed + 'Merged_Integrated_melt_rates.csv',index_col=0)
Paolo_Gt_ds = integrated_Paolo.to_xarray()

inputpath_Paolo_processed = '/bettik/millanr/DATA_SERVER/ANTARCTICA/OCEANICE/DATASET_FINAL/'
integrated_Paolo_unc = pd.read_csv(inputpath_Paolo_processed + 'Merged_Integrated_melt_err_rates.csv',index_col=0)
Paolo_Gt_unc_ds = integrated_Paolo_unc.to_xarray()


In [6]:
year_list = ['1992'] + np.arange(1994,2015).astype(str).tolist()
#year_list = np.arange(1992,2015).astype(str).tolist()
merged_list = []
merged_list_unc = []
for vvar in year_list:
    merged_list.append(Paolo_Gt_ds[vvar].rename('melt_Gt').assign_coords({'time': int(vvar)}))
    merged_list_unc.append(Paolo_Gt_unc_ds[vvar].rename('melt_Gt_unc').assign_coords({'time': int(vvar)}))
merged_ds = xr.concat(merged_list, dim='time')
merged_unc_ds = xr.concat(merged_list_unc, dim='time')
merged_ds_both = xr.merge([merged_ds,merged_unc_ds])

In [15]:
melt_Gt_list = []
for kisf in file_isf.Nisf:
    nname = file_isf['isf_name'].sel(Nisf=kisf).values
    if nname in merged_ds['Names']:
        melt_Gt_kisf = merged_ds_both.sel(Names=nname).squeeze().drop('Names')
    elif nname == 'Conger/Glenzer':
        melt_Gt_kisf = merged_ds_both.sel(Names='Conger_Glenzer').squeeze().drop('Names')
    elif nname == 'George VI':
        melt_Gt_kisf = merged_ds_both.sel(Names='George_VI').squeeze().drop('Names')
    elif str(nname)[0:7] == 'Larsen ':
        melt_Gt_kisf = merged_ds_both.sel(Names='Larsen'+str(nname)[7]).squeeze().drop('Names')
    elif nname == 'Prince Harald':
        melt_Gt_kisf = merged_ds_both.sel(Names='Prince_Harald').squeeze().drop('Names')
    elif nname == 'Moscow Univ.':    
        melt_Gt_kisf = merged_ds_both.sel(Names='Moscow_University').squeeze().drop('Names')
    elif nname == 'Roi Baudouin':
        melt_Gt_kisf = merged_ds_both.sel(Names='Baudouin').squeeze().drop('Names')
    elif nname == 'Wilma/Robert/Downer':
        melt_Gt_kisf = merged_ds_both.sel(Names='WilmaRobertDowner').squeeze().drop('Names')
    elif nname == 'Tracy Tremenchus':
        melt_Gt_kisf = merged_ds_both.sel(Names='Tracy_Tremenchus').squeeze().drop('Names')
    elif nname == 'Rayner/Thyer':
        melt_Gt_kisf = merged_ds_both.sel(Names='Rayner_Thyer').squeeze().drop('Names')
    elif nname == 'Pine Island':
        melt_Gt_kisf = merged_ds_both.sel(Names='Pine_Island').squeeze().drop('Names')
    elif nname == 'Stancomb Brunt':
        melt_Gt_kisf = merged_ds_both.sel(Names='Brunt_Stancomb').squeeze().drop('Names')
    elif nname == 'Edward VIII':
        melt_Gt_kisf = merged_ds_both.sel(Names='Edward_VIII').squeeze().drop('Names')
    elif nname == 'Vincennes Bay':
        melt_Gt_kisf = merged_ds_both.sel(Names='Vincennes_Bay').squeeze().drop('Names')
    elif nname == 'Filchner-Ronne':
        melt_abs = merged_ds_both['melt_Gt'].sel(Names='Ronne').squeeze().drop('Names') + merged_ds_both['melt_Gt'].sel(Names='Filchner').squeeze().drop('Names')
        melt_unc = np.sqrt(merged_ds_both['melt_Gt_unc'].sel(Names='Ronne').squeeze().drop('Names')**2 
                           + merged_ds_both['melt_Gt_unc'].sel(Names='Filchner').squeeze().drop('Names')**2)
        melt_Gt_kisf = xr.merge([melt_abs,melt_unc])
    elif nname == 'Ross':
        melt_abs = merged_ds_both['melt_Gt'].sel(Names='Ross_East').squeeze().drop('Names') + merged_ds_both['melt_Gt'].sel(Names='Ross_West').squeeze().drop('Names')
        melt_unc = np.sqrt(merged_ds_both['melt_Gt_unc'].sel(Names='Ross_East').squeeze().drop('Names')**2 
                           + merged_ds_both['melt_Gt_unc'].sel(Names='Ross_West').squeeze().drop('Names')**2)
        melt_Gt_kisf = xr.merge([melt_abs,melt_unc])
    
    elif nname == 'Wordie':
        melt_abs = (merged_ds_both['melt_Gt'].sel(Names='Wordie_(Harriott)').squeeze().drop('Names') 
                    + merged_ds_both['melt_Gt'].sel(Names='Wordie_(Harriott_Headland)').squeeze().drop('Names')
                    + merged_ds_both['melt_Gt'].sel(Names='Wordie_(Cape_Jeremy)').squeeze().drop('Names')
                    + merged_ds_both['melt_Gt'].sel(Names='Wordie_(Prospect)').squeeze().drop('Names')
                    + merged_ds_both['melt_Gt'].sel(Names='Wordie_(Airy_Rotz_Seller)').squeeze().drop('Names')
                   )
        melt_unc = np.sqrt(merged_ds_both['melt_Gt_unc'].sel(Names='Wordie_(Harriott)').squeeze().drop('Names')**2 
                    + merged_ds_both['melt_Gt_unc'].sel(Names='Wordie_(Harriott_Headland)').squeeze().drop('Names')**2 
                    + merged_ds_both['melt_Gt_unc'].sel(Names='Wordie_(Cape_Jeremy)').squeeze().drop('Names')**2 
                    + merged_ds_both['melt_Gt_unc'].sel(Names='Wordie_(Prospect)').squeeze().drop('Names')**2 
                    + merged_ds_both['melt_Gt_unc'].sel(Names='Wordie_(Airy_Rotz_Seller)').squeeze().drop('Names')**2 
                   )
        melt_Gt_kisf = xr.merge([melt_abs,melt_unc])
                    
    melt_Gt_list.append(melt_Gt_kisf.assign_coords({'Nisf': kisf}))

melt_Gt_all_Paolo = xr.concat(melt_Gt_list, dim='Nisf')
        


In [16]:
melt_Gt_all_Paolo.to_netcdf(outputpath_raw + 'basal_melt_Gt_Paolo23.nc')